# Liver Model Construction: Notebook-Glycolysis ONLY-trial

## Setup workflow

### Import packages

In [1]:
import os
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import sympy as sy
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass import MassMetabolite, MassModel, MassReaction
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile

print("MASSpy version: {0}".format(mass.__version__))

MASSpy version: 0.1.0b3


## Set paths and constants

### Directory paths

In [2]:
model_dir = os.path.abspath("../models")
maps_dir = os.path.abspath("../maps")

# Create MASS model from COBRA model

### Load COBRA model

In [3]:
core_model=load_json_cobra_model(filename=os.path.join(model_dir,"CoreModel.json"))
core_model

Name,None
Memory address,0x07fc7527a2990
Number of metabolites,178
Number of reactions,187
Number of groups,0
Objective expression,1.0*ATPM - 1.0*ATPM_reverse_5b752
Compartments,"m, i, c, r,"


## Create MASS model

In [4]:
glycolysis = MassModel("Glycolysis", array_type='DataFrame', dtype=np.int64)

### Define reactions

In [5]:
# glycolysis_map = escher.Builder(model=core_model,  map_json="Glycolysis, gluconeogenesis lactate fermentation 2.json",
#                                 highlight_missing=True) 
# glycolysis_map

In [6]:
reaction_list = ['HEX1',
                'PGI',
#                 'FBP',
                'PFK',
                'FBA',
                'TPI',
                'GAPD',
                'PGK',
                'PGM',
                'ENO',
#                 'PEPtm',
#                 'PEPCKm',
                 'LDH_L',
                'PYK']
#                 'PCm',
                 
#                  'PYRt2m']
new_reaction_list = []
for item in reaction_list: 
    item = core_model.reactions.get_by_id(item)
    new_reaction_list.append(item) 
    
new_reaction_list

# Convert cobra.Reactions to mass.MassReactions
for rid in reaction_list:
    reaction = core_model.reactions.get_by_id(rid)
    glycolysis.add_reactions([MassReaction(reaction)])
    
## set possible exchanges for in and out?




#### adding from masspy documentation

In [7]:
# new_reaction_list
m=["adp_c","amp_c", "atp_c","pi_c","nadh_c","nad_c", "h2o_c", "h_c"]
new_list = []
for item in m: 
    item = core_model.metabolites.get_by_id(item)
    new_list.append(item) 
print(new_list)

glycolysis

[<Metabolite adp_c at 0x7fc7527ae3d0>, <Metabolite amp_c at 0x7fc7527ae850>, <Metabolite atp_c at 0x7fc7527ae410>, <Metabolite pi_c at 0x7fc7527ae6d0>, <Metabolite nadh_c at 0x7fc7527ae690>, <Metabolite nad_c at 0x7fc7527ae650>, <Metabolite h2o_c at 0x7fc7527ae790>, <Metabolite h_c at 0x7fc7527ae4d0>]


Name,Glycolysis
Memory address,0x07fc7527343d0
Stoichiometric Matrix,19x11
Matrix Rank,11
Number of metabolites,19
Initial conditions defined,0/19
Number of reactions,11
Number of genes,11
Number of enzyme modules,0
Number of groups,0
Objective expression,0


In [8]:
glycolysis.metabolites

[<MassMetabolite adp_c at 0x7fc752702090>,
 <MassMetabolite atp_c at 0x7fc752702050>,
 <MassMetabolite g6p_c at 0x7fc7527020d0>,
 <MassMetabolite glc__D_c at 0x7fc752702110>,
 <MassMetabolite h_c at 0x7fc75272c310>,
 <MassMetabolite f6p_c at 0x7fc752734e90>,
 <MassMetabolite fdp_c at 0x7fc7527022d0>,
 <MassMetabolite dhap_c at 0x7fc74a406e90>,
 <MassMetabolite g3p_c at 0x7fc74a406e10>,
 <MassMetabolite 13dpg_c at 0x7fc752726e10>,
 <MassMetabolite nad_c at 0x7fc74a406ed0>,
 <MassMetabolite nadh_c at 0x7fc75396a950>,
 <MassMetabolite pi_c at 0x7fc75396ad50>,
 <MassMetabolite 3pg_c at 0x7fc75396ae50>,
 <MassMetabolite 2pg_c at 0x7fc75396a910>,
 <MassMetabolite h2o_c at 0x7fc753963150>,
 <MassMetabolite pep_c at 0x7fc753963510>,
 <MassMetabolite lac__L_c at 0x7fc75396aa50>,
 <MassMetabolite pyr_c at 0x7fc75395bc10>]

In [9]:
# Convert cobra.Reactions to mass.MassReactions
for rid in m:
    met = core_model.metabolites.get_by_id(rid)
    glycolysis.add_metabolites([MassMetabolite(met)])

In [10]:
glycolysis

Name,Glycolysis
Memory address,0x07fc7527343d0
Stoichiometric Matrix,20x11
Matrix Rank,11
Number of metabolites,20
Initial conditions defined,0/20
Number of reactions,11
Number of genes,11
Number of enzyme modules,0
Number of groups,0
Objective expression,0


In [11]:
glycolysis.metabolites

[<MassMetabolite adp_c at 0x7fc752702090>,
 <MassMetabolite atp_c at 0x7fc752702050>,
 <MassMetabolite g6p_c at 0x7fc7527020d0>,
 <MassMetabolite glc__D_c at 0x7fc752702110>,
 <MassMetabolite h_c at 0x7fc75272c310>,
 <MassMetabolite f6p_c at 0x7fc752734e90>,
 <MassMetabolite fdp_c at 0x7fc7527022d0>,
 <MassMetabolite dhap_c at 0x7fc74a406e90>,
 <MassMetabolite g3p_c at 0x7fc74a406e10>,
 <MassMetabolite 13dpg_c at 0x7fc752726e10>,
 <MassMetabolite nad_c at 0x7fc74a406ed0>,
 <MassMetabolite nadh_c at 0x7fc75396a950>,
 <MassMetabolite pi_c at 0x7fc75396ad50>,
 <MassMetabolite 3pg_c at 0x7fc75396ae50>,
 <MassMetabolite 2pg_c at 0x7fc75396a910>,
 <MassMetabolite h2o_c at 0x7fc753963150>,
 <MassMetabolite pep_c at 0x7fc753963510>,
 <MassMetabolite lac__L_c at 0x7fc75396aa50>,
 <MassMetabolite pyr_c at 0x7fc75395bc10>,
 <MassMetabolite amp_c at 0x7fc7527423d0>]

In [21]:
ADK1 = MassReaction(
    "ADK1",
    name="Adenylate kinase",
    subsystem="Misc.",
    reversible=True)

ADK = ['amp_c','atp_c']
for i in ADK:
    item = glycolysis.metabolites.get_by_id(i)
    ADK1.add_metabolites({item:1})
    
ADP =['adp_c']
for i in ADP:
    item = glycolysis.metabolites.get_by_id(i)
    ADK1.add_metabolites({item:-2})

ADK1




Reaction identifier,ADK1
Name,Adenylate kinase
Memory address,0x07fc753971ed0
Subsystem,Misc.
Kinetic Reversibility,True
Stoichiometry,2 adp_c <=> amp_c + atp_c 2 ADP C10H12N5O10P2 <=> AMP C10H12N5O7P + ATP C10H12N5O13P3
GPR,
Bounds,"(-1000.0, 1000.0)"


In [26]:
ATPM = MassReaction(
    "ATPM",
    name="ATP maintenance requirement",
    subsystem="Pseudoreaction",
    reversible=False)

ATPM_1 = ['atp_c', 'h2o_c']
ATPM_2= ['adp_c','h_c','pi_c']

for i in ATPM_1:
    item = glycolysis.metabolites.get_by_id(i)
    ATPM.add_metabolites({item:-1})
    

for i in ATPM_2:
    item = glycolysis.metabolites.get_by_id(i)
    ATPM.add_metabolites({item:1})

ATPM

Reaction identifier,ATPM
Name,ATP maintenance requirement
Memory address,0x07fc75383c150
Subsystem,Pseudoreaction
Kinetic Reversibility,False
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP C10H12N5O13P3 + H2O H2O --> ADP C10H12N5O10P2 + H+ + Phosphate
GPR,
Bounds,"(-1000.0, 1000.0)"


In [27]:


DM_nadh = MassReaction(
    "DM_nadh",
    name="Demand NADH",
    subsystem="Pseudoreaction",
    reversible=False)

DM_nadh_1 = ['nadh_c']
DM_nadh_2= ['nad_c','h_c']

for i in DM_nadh_1:
    item = glycolysis.metabolites.get_by_id(i)
    DM_nadh.add_metabolites({item:-1})
    

for i in DM_nadh_2:
    item = glycolysis.metabolites.get_by_id(i)
    DM_nadh.add_metabolites({item:1})

DM_nadh




Reaction identifier,DM_nadh
Name,Demand NADH
Memory address,0x07fc75383c510
Subsystem,Pseudoreaction
Kinetic Reversibility,False
Stoichiometry,nadh_c --> h_c + nad_c Nicotinamide adenine dinucleotide - reduced --> H+ + Nicotinamide adenine dinucleotide
GPR,
Bounds,"(-1000.0, 1000.0)"


In [30]:
# Add new rxns
glycolysis.add_reactions([ADK1, ATPM, DM_nadh])

for reaction in glycolysis.reactions:
    print(reaction)

HEX1: atp_c + glc__D_c <=> adp_c + g6p_c + h_c
PGI: g6p_c <=> f6p_c
PFK: atp_c + f6p_c <=> adp_c + fdp_c + h_c
FBA: fdp_c <=> dhap_c + g3p_c
TPI: dhap_c <=> g3p_c
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
PGM: 2pg_c <=> 3pg_c
ENO: 2pg_c <=> h2o_c + pep_c
LDH_L: lac__L_c + nad_c <=> h_c + nadh_c + pyr_c
PYK: adp_c + h_c + pep_c <=> atp_c + pyr_c
ADK1: 2 adp_c <=> amp_c + atp_c
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
DM_nadh: nadh_c --> h_c + nad_c


### View extracted network

In [33]:
### NEED TO CHECK!!
escher_builder = escher.Builder(
    model=glycolysis,
    map_json=os.path.join(
        maps_dir, ".".join((
            glycolysis.id, "map", "json"))
    ),
    highlight_missing=True)

escher_builder

ValueError: Could not load map_json. Not valid json, url, or filepath


### Set Initial Concentration

In [40]:
ID_list = ['glc__D_c',
           'g6p_c',
           'f6p_c',
           'fdp_c',
           'g3p_c',
           'dhap_c',
           '13dpg_c',
           '3pg_c',
            '2pg_c',
            'pep_c',
            'pyr_c',
            'lac__L_c',
            'atp_c',
            'adp_c',
            'amp_c',
            'h_c',
            'nad_c',
            'nadh_c',
            'pi_c',
            'h2o_c']
ID_conc = [0.010482807,
0.00014,
0.000127138,
5.15E-05,
8.78E-06,
0.000176897,
0.000243,
0.00052063,
0.000110561,
0.00031,
0.00048,
0.003261838,
0.004727146,
0.001994952,
0.000365,
9.00E-05,
0.00182531,
4.64E-06,
0.0064,
1]

#ID_dic = [{ID_list:ID_conc}]
print("Initial Conditions\n------------------")
for i in range(len(ID_list)):
    item = glycolysis.metabolites.get_by_id(ID_list[i])
    item.ic = ID_conc[i]
    print(item, ':', item.ic)
    



glc__D_c : 0.010482807
g6p_c : 0.00014
f6p_c : 0.000127138
fdp_c : 5.15e-05
g3p_c : 8.78e-06
dhap_c : 0.000176897
13dpg_c : 0.000243
3pg_c : 0.00052063
2pg_c : 0.000110561
pep_c : 0.00031
pyr_c : 0.00048
lac__L_c : 0.003261838
atp_c : 0.004727146
adp_c : 0.001994952
amp_c : 0.000365
h_c : 9e-05
nad_c : 0.00182531
nadh_c : 4.64e-06
pi_c : 0.0064
h2o_c : 1


In [ ]:
# print("Initial Conditions\n------------------")
# for metabolite, ic_value in glycolysis.initial_conditions.items():
#     print("{0}: {1}".format(metabolite, ic_value))

### Keq

In [41]:
reaction_list = ['HEX1',
                'PGI',
                'PFK',
                'FBA',
                'TPI',
                'GAPD',
                'PGK',
                'PGM',
                'ENO',
                 'LDH_L',
                'PYK',
                'ADK1',
                 'ATPM', 
                 'DM_nadh']
rxn_keq = [5000,
0.32,
1000,
0.1,
20.4,
0.005334127,
3200,
12.7,
4.7,
26300,
2220,
1,
1000000000,
1000000000]

print("Keq\n------------------")
for i in range(len(reaction_list)):
    item = glycolysis.metabolites.get_by_id(ID_list[i])
    item.ic = rxn_keq[i]
    print(item, ':', item.ic)

Keq
------------------
glc__D_c : 5000
g6p_c : 0.32
f6p_c : 1000
fdp_c : 0.1
g3p_c : 20.4
dhap_c : 0.005334127
13dpg_c : 3200
3pg_c : 12.7
2pg_c : 4.7
pep_c : 26300
pyr_c : 2220
lac__L_c : 1
atp_c : 1000000000
adp_c : 1000000000


### Boundary Reactons

In [45]:
SK_glc__D_c = glycolysis.add_boundary(
    metabolite=glycolysis.metabolites.get_by_id('glc__D_c'), boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)
SK_glc__D_c.reverse_stoichiometry(inplace=True)

Reaction identifier,SK_glc__D_c
Name,D-Glucose sink
Memory address,0x07fc753a6d9d0
Subsystem,Pseudoreaction
Kinetic Reversibility,True
Stoichiometry,<=> glc__D_c <=> D-Glucose
GPR,
Bounds,"(-1000.0, 1000.0)"


In [47]:


SK_lac__L_c = glycolysis.add_boundary(
    metabolite=glycolysis.metabolites.get_by_id('lac__L_c'), boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

SK_pyr_c = glycolysis.add_boundary(
    metabolite=glycolysis.metabolites.get_by_id('pyr_c'), boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=0.06)

SK_h_c = glycolysis.add_boundary(
    metabolite=glycolysis.metabolites.get_by_id('h_c'), boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=6.30957e-05)

SK_h2o_c = glycolysis.add_boundary(
    metabolite=glycolysis.metabolites.get_by_id('h2o_c'), boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)

SK_amp_c = glycolysis.add_boundary(
    metabolite=glycolysis.metabolites.get_by_id('amp_c'), boundary_type="sink", subsystem="Pseudoreaction",
    boundary_condition=1)
SK_amp_c.reverse_stoichiometry(inplace=True)

DM_amp_c = glycolysis.add_boundary(
    metabolite=glycolysis.metabolites.get_by_id('amp_c'), boundary_type="demand", subsystem="Pseudoreaction",
    boundary_condition=1)

print("Boundary Reactions and Values\n-----------------------------")
for reaction in glycolysis.boundary:
    boundary_met = reaction.boundary_metabolite
    bc_value = glycolysis.boundary_conditions.get(boundary_met)
    print("{0}\n{1}: {2}\n".format(
        reaction, boundary_met, bc_value))

Boundary Reactions and Values
-----------------------------
SK_glc__D_c:  <=> glc__D_c
glc__D_b: 1.0

SK_lac__L_c: lac__L_c <=> 
lac__L_b: 1.0

SK_pyr_c: pyr_c <=> 
pyr_b: 0.06

SK_h_c: h_c <=> 
h_b: 6.30957e-05

SK_h2o_c: h2o_c <=> 
h2o_b: 1.0

SK_amp_c:  <=> amp_c
amp_b: 1.0

DM_amp_c: amp_c --> 
amp_b: 1.0



### Fluxes

In [48]:
minspan_paths = [
    [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 1, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1, -1, 0, 1, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]]


In [49]:
glycolysis.compute_steady_state_fluxes(
    pathways=minspan_paths,
    independent_fluxes={
        SK_glc__D_c: 1.12,
        DM_nadh: .2 * 1.12,
        DM_amp_c: 0.014},
    update_reactions=True)

print("Steady State Fluxes\n-------------------")
for reaction, steady_state_flux in glycolysis.steady_state_fluxes.items():
    print("{0}: {1:.6f}".format(reaction.flux_symbol_str, steady_state_flux))


LinAlgError: Singular matrix

## Parameterize MASS model

In [ ]:
##possible functions
# loading equilibrium constants/ fluxes, initial conditions
# potential conversion factors?
# enzyme modules?


In [ ]:
## flux split through pathways??

In [ ]:
## making enzyme modules??


#### Set fluxes

In [ ]:
## load independent fluxes
# minspan paths, independent fluxes
# steady state fluxes
# print everything

#### Set equilibrium constants

In [ ]:
# read Keq
# set Keq
# print Keqs

### Obtain Concentrations

#### Set concentrations

In [ ]:
# Provide initial guesses for a few, setting?
# load, set, print conc values

In [ ]:
## potential optimal solutions? using QP and then comparing

In [ ]:
## calculate PERC's

In [ ]:
## import/merging enzyme modules into model

## Simulate at steady state

# Export MASS model

In [ ]:
save_json_mass_model(
    mass_model=glycolysis,
    filename=os.path.join(model_dir, glycolysis.id + ".json"))